# Architecture Review: Cifar10 with PyTorch
![image](output.png)
## Introduction
This notebook is a review of the architecture used in the PyTorch tutorial for the CIFAR10 dataset. This is the complete code for the tutorial, with some minor modifications to make it more readable and to modify the architectures for this dataset. To find the practice notebook, please visit the following link: [Practice Notebook](). 

### Pre-requisites
This notebook assumes that you have a basic understanding of neural networks and PyTorch. If you are new to PyTorch, please visit the following link to get started: [PyTorch Tutorials](https://pytorch.org/tutorials/). Please complete the [MNIST tutorial](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html) before starting this tutorial.

### Dataset
The CIFAR10 dataset consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. There are 50,000 training images and 10,000 test images. The classes are completely mutually exclusive. There is no overlap between automobiles and trucks. "Automobile" includes sedans, SUVs, and other similar vehicles. "Truck" includes only big trucks. The test batch contains 1,000 randomly selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5,000 images from each class. While you may realize that this doesn't sound like it's useful for real-world applications, it's a good dataset to start with and learn the basics of deep learning since it's small and easy to work with. 

**Classes**:
- airplane
- automobile
- bird
- cat
- deer
- dog
- frog
- horse
- ship
- truck
- [CIFAR10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html)
- [CIFAR10 Dataset PyTorch](https://pytorch.org/vision/stable/datasets.html#cifar)

**Critcism**:
The CIFAR10 dataset is a very small dataset and is not representative of real-world data. It is used for educational purposes and to learn the basics of deep learning. Intersting read: [Once Upon a Time in CIFAR-10](https://franky07724-57962.medium.com/once-upon-a-time-in-cifar-10-c26bb056b4ce#:~:text=However%2C%20the%20quality%20of%20CIFAR,10%20contains%200.54%25%20label%20errors.)


**Note**: 
There is also a CIFAR100 dataset, which has 100 classes. For the extra challenge, you can try working with the CIFAR100 dataset.

---

## Code setup
Let's start by importing the necessary libraries and setting up the code for the tutorial.

In [1]:
import torch
import torch.nn as nn
import torchvision as tv
import tqdm as tqdm
from lightning import pytorch as pl
from pytorch_lightning.loggers import WandbLogger


Then we need to load the CIFAR10 dataset using the `torchvision` library. The dataset is divided into training and testing sets, and each set contains images and their corresponding labels (0-9). The images are also normalized to have a mean of 0.5 and a standard deviation of 0.5. This normalization helps the neural network since it allows the weights to be updated more evenly during training.

The batch size is set to 64, though you can change it to a different value if you like. The batch size determines how many images are processed at once during training. A larger batch size can speed up training but requires more memory. A smaller batch size can slow down training but may lead to better generalization.

Finally, we create a DataLoader for the training and testing sets. The DataLoader is used to load the data in batches during training and testing. It also shuffles the data during training to ensure that the model learns from different samples in each epoch.

In [2]:
class CIFAR10Classifier(pl.LightningModule):
    def __init__(
        self,
        model: nn.Module,
        batch_size: int,
        lr: float,
        w_decay: float,
        T_max: int,
    ):
        super().__init__()
        self.model = model

        self.batch_size = batch_size
        self.lr = lr
        self.w_decay = w_decay
        self.T_max = T_max

        self.criterion = nn.CrossEntropyLoss()

        self.save_hyperparameters()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        
        train_acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("train_acc", train_acc)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        
        val_acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("val_acc", val_acc)
        
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        
        test_acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("test_acc", test_acc)
        
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=self.lr,
            weight_decay=self.w_decay,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=self.T_max
        )
        return [optimizer], [scheduler]

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            tv.datasets.CIFAR10(
                root="./data",
                train=True,
                download=True,
                transform=tv.transforms.ToTensor(),
            ),
            batch_size=self.batch_size,
            shuffle=True,
        )
        
    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            tv.datasets.CIFAR10(
                root="./data",
                train=False,
                download=True,
                transform=tv.transforms.ToTensor(),
            ),
            batch_size=self.batch_size,
            shuffle=False,
        )
        
    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            tv.datasets.CIFAR10(
                root="./data",
                train=False,
                download=True,
                transform=tv.transforms.ToTensor(),
            ),
            batch_size=self.batch_size,
            shuffle=False,
        )


## Model Architectures

In [3]:
hyperparameters = {
    "batch_size": 64,
    "lr": 2e-4,
    "w_decay": 5e-4,
    "T_max": 200,
}


### Linear Model

#### Single Layer Perceptron

In [5]:
if __name__ == "__main__":
    # Define a simple model
    model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(32 * 32 * 3, 10),
    )
    
    logger = WandbLogger(
        name="single-layer-perceptron",
        project="cifar10",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    cifarTrainer = CIFAR10Classifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=100,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    trainer.fit(cifarTrainer)
    trainer.test(cifarTrainer)

    logger.experiment.finish()


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'model' removed from hparams because it cannot be pickled

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 30.7 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
30.7 K    Trainable params
0         Non-trainable params
30.7 K    Total params
0.123     T

Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=100` reached.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.4043000042438507     │
│         test_loss         │    1.7288148403167725     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_acc,▁
test_loss,▁
train_acc,▁▅▅▅▅▄▄▇▄▄▅▄▆▆▂▄▅▃▃██▄▂█▆▅▅▄▆▄▅▅▆▅▆▄▇▆▆▇
train_loss,█▄▄▄▄▆▄▄▄▅▄▆▃▁▆▆▇▅▆▁▂▃▅▁▄▄▄▅▂▄▅▄▄▃▃▅▃▄▄▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▅▄▅▆▆▇█▇▇█▆▇▇▇▇▆▇▇▇▇▇▇▆▇█▇▇▇█▇▆▇▇▇▇▇▆██
val_loss,█▅▄▃▂▃▂▂▂▂▁▂▂▁▂▂▂▂▂▂▁▁▂▂▂▂▂▂▁▁▂▂▁▁▁▂▂▂▁▂
epoch,100
test_acc,0.4043
test_loss,1.72881


#### Multi Layer Perceptron

In [6]:
if __name__ == "__main__":
    # Define a simple model
    model = torch.nn.Sequential(
        # Flatten Layer
        torch.nn.Flatten(),
        # Fully Connected Layer
        torch.nn.Linear(32 * 32 * 3, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 10),
    )
    
    logger = WandbLogger(
        name="multi-layer-perceptron",
        project="cifar10",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    cifarTrainer = CIFAR10Classifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=100,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    trainer.fit(cifarTrainer)
    trainer.test(cifarTrainer)

    logger.experiment.finish()


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'model' removed from hparams because it cannot be pickled

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 1.7 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.829     T

Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=100` reached.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5376999974250793     │
│         test_loss         │    2.2734217643737793     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_acc,▁
test_loss,▁
train_acc,▁▂▃▃▄▄▄▅▆▅▃▄▅▅▅▅▆▅▆▆▆▅▇▇▆▇▆▇▇▇▆█▇▇▇███▇█
train_loss,█▇▆▇▆▅▅▅▄▅▆▅▄▄▃▄▃▄▃▃▃▄▃▂▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▄▄▆▆▇▆▇▇▇▇▇█████▇▇▇███▇▇███▇▇▇▇▇▇▇█▇▇▇▇
val_loss,▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▆▆▆▆▇▇██
epoch,100
test_acc,0.5377
test_loss,2.27342


### LeNet-5

![lenet-5](https://www.datasciencecentral.com/wp-content/uploads/2021/10/1lvvWF48t7cyRWqct13eU0w.jpeg)

#### Normal

In [8]:
if __name__ == "__main__":
    model = nn.Sequential(
        # First convolutional layer
        nn.Conv2d(3, 6, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # Second convolutional layer
        nn.Conv2d(6, 16, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # Flatten layer
        nn.Flatten(),
        # Fully connected layers
        nn.Linear(16 * 5 * 5, 120),
        nn.ReLU(),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Linear(84, 10),
    )
    
    logger = WandbLogger(
        name="lenet-5",
        project="cifar10",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    cifarTrainer = CIFAR10Classifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=100,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    trainer.fit(cifarTrainer)
    trainer.test(cifarTrainer)

    logger.experiment.finish()


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'model' removed from hparams because it cannot be pickled

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 62.0 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
62.0 K    Trainable params
0         Non-trainable params
62.0 K    Total params
0.248     T

Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=100` reached.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6301000118255615     │
│         test_loss         │    1.1256043910980225     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_acc,▁
test_loss,▁
train_acc,▁▃▂▂▄▂▄▄▃▄▄▄▅▅▅▄▆▆▅▅▆▅▅▇▂▅▅▄▄▆▄▆▇▅▇█▆▇▇▆
train_loss,███▆▆▆▄▅▅▄▅▅▄▄▃▅▃▃▄▄▃▃▄▂▆▃▄▃▅▃▄▃▂▃▁▁▂▂▁▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▃▅▅▆▆▆▆▇▇▇▇▇▇█▇▇███████████████████████
val_loss,█▆▄▄▄▃▃▃▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁
epoch,100
test_acc,0.6301
test_loss,1.1256


#### With Dropout (Regularization)

In [9]:
if __name__ == "__main__":
    model = nn.Sequential(
        # First convolutional layer
        nn.Conv2d(3, 6, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # Second convolutional layer
        nn.Conv2d(6, 16, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # Flatten layer
        nn.Flatten(),
        # Dropout Layer
        nn.Dropout(0.5),
        # Fully connected layers
        nn.Linear(16 * 5 * 5, 120),
        nn.ReLU(),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Linear(84, 10),
    )
    
    logger = WandbLogger(
        name="lenet-5 | dropout",
        project="cifar10",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    cifarTrainer = CIFAR10Classifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=100,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    trainer.fit(cifarTrainer)
    trainer.test(cifarTrainer)

    logger.experiment.finish()


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'model' removed from hparams because it cannot be pickled

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 62.0 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
62.0 K    Trainable params
0         Non-trainable params
62.0 K    Total params
0.248     T

Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.4977000057697296     │
│         test_loss         │    1.3893680572509766     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█
test_acc,▁
test_loss,▁
train_acc,▁▃▃▅▅▅▆▇▆▆▆▅▆▆▆▆▅▇▅▇▅█▆▇▇▇██▇▇▇▆▆▆▆▆▇▆█▇
train_loss,█▆▅▅▄▅▃▃▃▃▃▅▄▃▃▄▃▂▄▁▃▂▂▃▂▃▂▂▃▁▃▂▂▃▃▂▂▂▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▃▄▄▅▅▅▅▆▆▆▇▇▇▇█████
val_loss,█▇▅▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
epoch,10
test_acc,0.4977
test_loss,1.38937


### More Modern CNNs and better Training Techniques

In [10]:
if __name__ == "__main__":
    model = nn.Sequential(
        # First convolutional layer
        nn.Conv2d(3, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(32, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Second convolutional layer
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Third convolutional layer
        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Flatten layer
        nn.Flatten(),

        # Fully connected layers
        nn.Linear(128 * 4 * 4, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 10)
    )
    
    logger = WandbLogger(
        name="cnn",
        project="cifar10",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    cifarTrainer = CIFAR10Classifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=100,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    trainer.fit(cifarTrainer)
    trainer.test(cifarTrainer)

    logger.experiment.finish()


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'model' removed from hparams because it cannot be pickled

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 1.3 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.365     T

Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=100` reached.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7817999720573425     │
│         test_loss         │    1.8815202713012695     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_acc,▁
test_loss,▁
train_acc,▁▂▃▅▅▅▆▆▇▇▇▇▇▇▇▇█▇█▇▇█████▇█████████████
train_loss,█▆▆▄▄▃▃▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▃▅▆▇▇▇█▇▇█▇████████████████████████████
val_loss,▆▄▃▂▁▁▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▅▅▆▅▅▇▆▆▇▆▇▇▇▇▇▇█▇█
epoch,100
test_acc,0.7818
test_loss,1.88152


#### Batch Normalization + Dropout (Regularization)

In [13]:
if __name__ == "__main__":
    model = nn.Sequential(
    # First convolutional layer
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Second convolutional layer
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Third convolutional layer
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Flatten layer
    nn.Flatten(),
    # Fully connected layers
    nn.Linear(128 * 4 * 4, 512),
    nn.ReLU(),
    nn.Linear(512, 10)
)
    
    logger = WandbLogger(
        name="cnn | batchnorm",
        project="cifar10",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    cifarTrainer = CIFAR10Classifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=100,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    trainer.fit(cifarTrainer)
    trainer.test(cifarTrainer)

    logger.experiment.finish()



wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'model' removed from hparams because it cannot be pickled

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 1.3 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.368     T

Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=100` reached.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8140000104904175     │
│         test_loss         │     1.535017967224121     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_acc,▁
test_loss,▁
train_acc,▁▄▄▇▇▇▇█▇███████████████████████████████
train_loss,█▅▅▂▂▂▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▆▆▇▇▇▇▇▇▇▇▇▇███▇▇▇▇▆█▇▇█▇▇▇██▇██▇███▇██
val_loss,▃▁▁▁▂▂▃▃▃▃▃▄▄▄▄▄▅▄▅▅▆▅▅▆▅▆▅▆▅▆▆▆▅▆▆▆▆█▇▆
epoch,100
test_acc,0.814
test_loss,1.53502


In [14]:
if __name__ == "__main__":
    model = nn.Sequential(
    # First convolutional layer
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Second convolutional layer
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Third convolutional layer
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Flatten layer
    nn.Flatten(),
    # Fully connected layers
    nn.Dropout(0.5),
    nn.Linear(128 * 4 * 4, 512),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(512, 10)
)
    
    logger = WandbLogger(
        name="cnn | batchnorm | dropout",
        project="cifar10",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    cifarTrainer = CIFAR10Classifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=100,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    trainer.fit(cifarTrainer)
    trainer.test(cifarTrainer)

    logger.experiment.finish()



wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'model' removed from hparams because it cannot be pickled

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 1.3 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.368     T

Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=100` reached.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8418999910354614     │
│         test_loss         │     0.96767258644104      │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_acc,▁
test_loss,▁
train_acc,▁▄▅▃▆▆▇▇▆▇▇▇▇▇▇▇█▇▇█▇██████▇████████████
train_loss,█▅▄▆▃▃▂▂▃▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▄▆▆▇▇▆▆▇▇█▇▇▇▇▇▇▇█▇█▇▇█▇███▇▇███▇████▇█
val_loss,▇▃▁▁▁▁▂▃▃▂▃▃▄▅▃▅▅▄▄▆▄▅▇▅▅▆▅▆▇█▆▆▇▆▆▇██▇█
epoch,100
test_acc,0.8419
test_loss,0.96767


### VGG-16

![vgg-16](https://miro.medium.com/v2/resize:fit:1400/1*NNifzsJ7tD2kAfBXt3AzEg.png)

In [15]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = self._make_layers([64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'])
        self.classifier = nn.Linear(512, 10)

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

if __name__ == "__main__":
    model = VGG()
    
    logger = WandbLogger(
        name="vgg-16",
        project="cifar10",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    cifarTrainer = CIFAR10Classifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=20,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    trainer.fit(cifarTrainer)
    trainer.test(cifarTrainer)

    logger.experiment.finish()


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'model' removed from hparams because it cannot be pickled

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | VGG              | 14.7 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
14.7 M    Trainable params
0         Non-trainable params
14.7 M    Total params
58.913    T

Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


Files already downloaded and verified


/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8119999766349792     │
│         test_loss         │    0.6219227313995361     │
└───────────────────────────┴───────────────────────────┘

### ResNet-50

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [ ]:
if __name__ == "__main__":
    model = ResNet(Bottleneck, [2, 2, 2, 2]) # ResNet-18
    
    logger = WandbLogger(
        name="resnet-50",
        project="cifar10",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    cifarTrainer = CIFAR10Classifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=20,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    trainer.fit(cifarTrainer)
    trainer.test(cifarTrainer)

    logger.experiment.finish()


In [ ]:
if __name__ == "__main__":
    model = ResNet(Bottleneck, [3, 4, 6, 3]) # ResNet-50
    
    logger = WandbLogger(
        name="resnet-50",
        project="cifar10",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    cifarTrainer = CIFAR10Classifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=20,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    trainer.fit(cifarTrainer)
    trainer.test(cifarTrainer)

    logger.experiment.finish()
